### This notebook demonstrates how to update sql tables using Natural Language



In [13]:
# Install Packages
import openai
import pyodbc
import pandas as pd
import time
import json

First, we set up Azure OpenAI and SQL Server

In [15]:
def setup_configuration():   

    with open(r'config.json') as config_file:
        config_details = json.load(config_file)

    API_KEY = config_details['OPENAI_API_KEY']  
    RESOURCE_ENDPOINT = config_details['OPENAI_API_BASE']
    API_VERSION = "2023-05-15"
    
    return API_KEY, RESOURCE_ENDPOINT, API_VERSION

def run_sql_command(sql_upate): 

    with open(r'config.json') as config_file:
        config_details = json.load(config_file)

    database = config_details['DATABASE']  
    server = config_details['SERVER']   
    username = config_details['USER']  
    password = config_details['PASSWORD']  

    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    cursor.execute(sql_upate)
    cnxn.commit()

openai.api_type = "azure"
openai.api_key = setup_configuration()[0]
openai.api_base = setup_configuration()[1]
openai.api_version = setup_configuration()[2]


# Prompt Engineering

In [16]:
def generate_nl_to_sql(prompt_in, history = []):
    '''
    This GPT engine is setup for NLtoSQL tasks on the Sales DB.
    Input: NL question related to sales
    Output: SQL update command to run on the sales database
    '''

    messages=[
            {"role": "system", "content": """ 
            You are a SQL programmer Assistant.
            Your role is to generate SQL code (SQL Server)
            Never engage with user.
            Only respond with plain SQL code            
            Make sure to disambiguate column names when creating sql update commands that use more than one table. 
            If a valid SQL update command cannot be generated, only say "ERROR:" followed by why it cannot be generated.

                  Use the following sales database schema to write SQL queries:
                  Customers(cust_id INTEGER, cust_name VARCHAR, cust_email VARCHAR, cust_phone VARCHAR, cust_address VARCHAR, PRIMARY KEY (cust_id))
                  Products(prod_id INTEGER,prod_name varchar, price FLOAT, category VARCHAR, PRIMARY KEY(prod_id))
                  Stock(prod_id INTEGER, merchant_id INTEGER, stock INTEGER, PRIMARY KEY(prod_id, merchant_id), FOREIGN KEY(merchant_id, prod_id))
                  Merchants(merchant_id INTEGER, merchant_name VARCHAR, merchant_region VARCHAR, merchant_address VARCHAR, PRIMARY KEY(merchant_id))
                  Sales(sale_id INTEGER, cust_id INTEGER , merchant_id INTEGER , date TIMESTAMP, total_price FLOAT, PRIMARY KEY(sale_id),FOREIGN KEY(cust_id,merchant_id))
                  Sales_Detail(sales_id INTEGER, prod_id INTEGER, quantity INTEGER, PRIMARY KEY(sales_id,prod_id), FOREIGN KEY(sales_id,prod_id))

                  Examples:
                  User:for customers with id=10 modify phone number to 555-555-5555
                  Assistant : UPDATE Customers SET cust_phone = '555-555-5555' WHERE cust_id = 10;
                  User: I see some products have invali price.  set price equals 0
                  Assistant: UPDATE Products SET price = 0 WHERE price < 0;
                  User: modify sales price that is not numerical.  assign the value 0
                  Assistant: UPDATE Sales SET total_price = 0 WHERE ISNUMERIC(total_price) = 0;

                """}
        ]

    messages.extend(history)
    messages.append({"role": "user", "content": prompt_in})
    
    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", 
        messages=messages,
        temperature=0,
        max_tokens=2000,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return response['choices'][0]['message']['content']

# Examples

Our user input is the NL question, as an output, Azure OpenAI generates as completion the SQL update commands we should run on the Sales database 

In [19]:
question_products = 'for customers with id=10 modify phone number to 111-555-5555'
completion_products = generate_nl_to_sql(question_products)
print(completion_products)

UPDATE Customers SET cust_phone = '111-555-5555' WHERE cust_id = 10;


We run the SQL query on our database. We retrive the information as a pandas dataframe

In [20]:
sql_output_products = run_sql_command(completion_products)
print(sql_output_products)

None


In [10]:
question_products = 'for merchant replace Blvd witn Boulevard'
completion_products = generate_nl_to_sql(question_products)
print(completion_products)

UPDATE Merchants SET merchant_address = REPLACE(merchant_address, 'Blvd', 'Boulevard');


In [11]:
sql_output_products = run_sql_command(completion_products)
print(sql_output_products)

None
